In [ ]:
# BLANK
import pathlib
from datetime import datetime
import time

import torch
from torch import cuda

import glob
import json
import os
import shutil
import sys
import ast
import random
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import skimage.io as io
from shapely import Polygon
from matplotlib import pyplot as plt
from PIL import Image
from torch import cuda
import supervision as sv
from ultralytics import YOLO
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm, trange
from importlib import reload

# Clone georip repo here: https://www.github.com/joeletho/georip.git

# Cloned repo directory
sys.path.append("path/to/georip")

import georip


In [ ]:
# BLANK
print(sys.version)

In [ ]:
# BLANK
print(reload(georip))


In [ ]:
# BLANK
#
has_gpu = cuda.is_available()

device = torch.device('cuda' if has_gpu else 'cpu')
print(device)
if has_gpu:
    print(cuda.get_device_name(0))

In [ ]:
# BLANK
path_map = {}

# Change this to your project path
path_map['ROOT'] = Path("path/to/project/root")

path_map['PROJECT_NAME'] = 'GEORIP_YOLO'
path_map['georip'] = path_map['ROOT'] / path_map["PROJECT_NAME"]
path_map['GEORIP_DS'] = path_map['georip'] / 'datasets'
path_map['GEORIP_MODELS'] = path_map['georip'] / 'models'

In [ ]:
# BLANK
def make_directories(paths_map, verbose=True, exist_ok=False):
    if verbose:
        print("Creating directory structure")
    for name, path in paths_map.items():
        if isinstance(path, Path):
            if path.is_file():
                paths_map[name] = path.resolve()
            else:
                path = path.resolve()
                paths_map[name] = path
                path.mkdir(parents=True, exist_ok=exist_ok)
                if verbose:
                    print('  ',path)
    if verbose:
        print("Complete")

def make_project_paths(root,*, verbose=True, exist_ok=False):    
    paths = {'NDVI': Path(root, 'NDVI', 'NDVI Difference Rasters')}
    paths['SHAPE_FILES'] = Path(root, 'Shapefiles')

    paths['GEORIP_DS_META'] = path_map['GEORIP_DS'] / 'meta'
    paths['GEORIP_DS_CSV'] = paths['GEORIP_DS_META'] / 'csv'
    paths['GEORIP_DS_SHP'] = paths['GEORIP_DS_META'] / 'shp'
    
    # Data
    paths['PRED_SHP'] = paths['SHAPE_FILES'] / 'ModelPredictions'
    paths['SHPZ10_SHP'] = paths['SHAPE_FILES'] / 'Treatments_UTMz10_Only_08-18-24' / 'Treatments_UTMz10_Only_08-18-24.shp'
    paths['SHPZ11_SHP'] = paths['SHAPE_FILES'] / 'Treatments_UTMz11_Only_08-18-24' / 'Treatments_UTMz11_Only_08-18-24.shp'
    for name, path in paths.items():
        path_map[name] = path
    make_directories(paths, verbose=verbose, exist_ok=exist_ok)

make_project_paths(path_map['ROOT'], exist_ok=True)

In [ ]:
# BLANK
DS = path_map['GEORIP_DS']

total_updates = len(paths)*2 # train & val for each dataset
root_pbar = trange(total_updates, desc="Drawing datasets")
updates = 0

for path in paths:
    print(path)
    for ds_type in ['train','val']:
        if path.name == 'meta':
            continue
        ds = georip.datasets.YOLODatasetLoader(
            classes=['Treatment'],
            images_dir=path / f'images/{ds_type}', 
            annotations_dir=path / f'labels/{ds_type}',
        )
        ds.draw(
            save_dir=path / 'images/annotated', 
            show=True, 
            include_background=False,
            verbose=False,
            pbar=True,
            pbar_desc=f"{path.name} - {ds_type}"
        )
        
        root_pbar.update()
        updates += 1

if updates < total_updates:
    root_pbar.update(total_updates - updates)
root_pbar.close()
                    

In [ ]:
# 